In [1]:
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
url = "https://www.magicbricks.com/property-for-rent/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Service-Apartment&cityName=Hyderabad&BudgetMin=5000&BudgetMax=50000&page="

In [2]:
def house_price_scraper(url,headers):
    #count = 0
    request = Request(url, headers=headers)
    response = urlopen(request)
    html = response.read()
    html_soup = BeautifulSoup(html , 'html.parser')
    house_container = html_soup.find_all('div', class_ = 'flex relative clearfix m-srp-card__container' )
    for container in range(len(house_container)) :
        #print(count)
        first  = house_container[container].find_all('span')
        second = house_container[container].find_all('div',class_ = 'm-srp-card__summary__info')
        #Price of the Flat
        Price_  = first[1].text.replace('\n','')
        Price.append((Price_))
        #No of Bedrooms
        Bedrooms_   = first[3].text.replace('\n','')
        Bedrooms.append((Bedrooms_))
        #No of Bathrooms
        try:
            Bathrooms_  = second[4].text.replace('\n','')
            Bathrooms.append(Bathrooms_)
        except :
            Bathrooms.append('NaN')
        # Area of the house in Sqft
        try:
            Area_       = first[4].text.replace('\n','')
            Area.append((Area_))
        except:
            Area.append(('NaN'))
        # Type of Furnishing i.e Semi-Furnished or Fully Furnished
        Furnishing_ = second[1].text.replace('\n','')
        Furnishing.append(Furnishing_)
        # Preferred Tennants i.e Bachelors or Family or Both
        try:
            Tennants_   = second[2].text.replace('\n','')
            Tennants.append(Tennants_)
        except:
            Tennants.append('NaN')
        #Locality where the Flat is located
        Locality_0  = house_container[container].find_all('span')[2]
        Locality_1 = Locality_0.text.replace('\n',' ')
        Locality_2 = re.search(r'in(.+?)\d', Locality_1)
        if Locality_2 is None :
            Locality_2 = re.search(r'in(.+?),',Locality_1)
        if Locality_2 is None :
            Locality_2 = re.search(r'in(.+?) ',Locality_1)
        Locality_  = Locality_2[1]
        Locality.append(Locality_)
        #print((Locality_))
        
        
    # Making the Data frame out of the scraped columns
    cols = ['Bedrooms', 'Bathrooms', 'Furnishing', 'Tennants', 'Area', 'Price','Locality']

    house_data = pd.DataFrame({'Bedrooms': Bedrooms,
                               'Bathrooms': Bathrooms,
                               'Furnishing': Furnishing,
                               'Tennants': Tennants,
                               'Area': Area,
                               'Price': Price,
                               'Locality':Locality})[cols]
    return house_data

In [3]:
Bedrooms = []
Bathrooms = []
Furnishing = []
Tennants = []
Locality = []
Area = []
Price = []

In [4]:
for i in range(1,40):
    url_ = url+str(i)
    house_data = house_price_scraper(url_,headers)

In [5]:
house_data.count()

Bedrooms      1171
Bathrooms     1171
Furnishing    1171
Tennants      1171
Area          1171
Price         1171
Locality      1171
dtype: int64

In [6]:
house_data.to_csv('magics_brics_data.csv')